In [1]:
# importing the usual things
import numpy as np
import pandas as pd
from matplotlib import pyplot as plt
import os
import json
import random

# importing visual and 3D data packages
import cv2
import open3d as o3d

# importing geoprocessing packages
from pymap3d import enu2geodetic
import gdal, osr

# importing useful opensfm functions
import sys
# insert at 1, 0 is the script path (or '' in REPL)
sys.path.insert(1, '/home/ragarcia/OpenSfM/')
from opensfm.features import denormalized_image_coordinates

## Running the reconstruction
Switching here to command line

In [4]:
# General prerequisites for doing reconstruction
!opensfm extract_metadata projects/mercedita

!opensfm detect_features projects/mercedita

!opensfm match_features projects/mercedita

2020-09-02 13:36:20,288 INFO: Extracting EXIF for DSC_0013_0ff205ae-e44f-4e64-93ef-9486c9bd0af6.jpg
2020-09-02 13:36:20,372 INFO: Extracting EXIF for DSC_0009_86d6a9c1-bf1f-4a60-bd78-0314108ad8de.jpg
2020-09-02 13:36:20,451 INFO: Extracting EXIF for DSC_0024_9c2f518f-be72-4cfb-94f3-3b703c50339d.jpg
2020-09-02 13:36:20,527 INFO: Extracting EXIF for DSC_0003_43058503-bac4-4655-971b-66374c69d2a7.jpg
2020-09-02 13:36:20,611 INFO: Extracting EXIF for DSC_0025_75937cbe-1197-4b32-85c6-01888538517c.jpg
2020-09-02 13:36:20,688 INFO: Extracting EXIF for DSC_0010_3ef2666f-35ae-4ed9-a3f2-e9cebaa96921.jpg
2020-09-02 13:36:20,772 INFO: Extracting EXIF for DSC_0028_ecb7bab7-d7bd-4e03-b424-2aa624531942.jpg
2020-09-02 13:36:20,852 INFO: Extracting EXIF for DSC_0006_c4295777-fab7-43be-8f1a-2b1c402d0b40.jpg
2020-09-02 13:36:20,932 INFO: Extracting EXIF for DSC_0027_28fd12bf-d71a-4546-9bb4-6cfb08bce33d.jpg
2020-09-02 13:36:21,007 INFO: Extracting EXIF for DSC_0012_da76f6eb-f7f8-4b55-ad99-038e95dbc4f7.jpg


2020-09-02 13:36:48,668 DEBUG: Matching DSC_0006_c4295777-fab7-43be-8f1a-2b1c402d0b40.jpg and DSC_0004_34191577-30cf-40c1-a6b0-d3d84796316d.jpg.  Matcher: FLANN (symmetric) T-desc: 0.402 T-robust: 0.000 T-total: 0.403 Matches: 97 Robust: 89 Success: True
2020-09-02 13:36:48,677 DEBUG: No segmentation for DSC_0009_86d6a9c1-bf1f-4a60-bd78-0314108ad8de.jpg, no features masked.
2020-09-02 13:36:48,950 DEBUG: Matching DSC_0006_c4295777-fab7-43be-8f1a-2b1c402d0b40.jpg and DSC_0009_86d6a9c1-bf1f-4a60-bd78-0314108ad8de.jpg.  Matcher: FLANN (symmetric) T-desc: 0.274 T-robust: 0.000 T-total: 0.274 Matches: 122 Robust: 109 Success: True
2020-09-02 13:36:48,960 DEBUG: No segmentation for DSC_0008_c68f493f-5f8f-449d-a6ba-039948ceb1ec.jpg, no features masked.
2020-09-02 13:36:49,221 DEBUG: Matching DSC_0006_c4295777-fab7-43be-8f1a-2b1c402d0b40.jpg and DSC_0008_c68f493f-5f8f-449d-a6ba-039948ceb1ec.jpg.  Matcher: FLANN (symmetric) T-desc: 0.261 T-robust: 0.001 T-total: 0.262 Matches: 431 Robust: 415 S

2020-09-02 13:36:53,976 DEBUG: Matching DSC_0009_86d6a9c1-bf1f-4a60-bd78-0314108ad8de.jpg and DSC_0012_da76f6eb-f7f8-4b55-ad99-038e95dbc4f7.jpg.  Matcher: FLANN (symmetric) T-desc: 0.134 T-robust: 0.000 T-total: 0.135 Matches: 140 Robust: 132 Success: True
2020-09-02 13:36:54,113 DEBUG: Matching DSC_0009_86d6a9c1-bf1f-4a60-bd78-0314108ad8de.jpg and DSC_0010_3ef2666f-35ae-4ed9-a3f2-e9cebaa96921.jpg.  Matcher: FLANN (symmetric) T-desc: 0.132 T-robust: 0.003 T-total: 0.135 Matches: 1707 Robust: 1676 Success: True
2020-09-02 13:36:54,251 DEBUG: Matching DSC_0009_86d6a9c1-bf1f-4a60-bd78-0314108ad8de.jpg and DSC_0013_0ff205ae-e44f-4e64-93ef-9486c9bd0af6.jpg.  Matcher: FLANN (symmetric) T-desc: 0.137 T-robust: 0.001 T-total: 0.137 Matches: 51 Robust: 41 Success: True
2020-09-02 13:36:54,390 DEBUG: Matching DSC_0009_86d6a9c1-bf1f-4a60-bd78-0314108ad8de.jpg and DSC_0007_e7785cc7-f4de-4715-8071-a4a993fba9f1.jpg.  Matcher: FLANN (symmetric) T-desc: 0.137 T-robust: 0.001 T-total: 0.138 Matches: 48

In [16]:
# Run this cell if you want to partition the model into separate submodels (remember to change parameters in the config file)
!opensfm create_submodels projects/mercedita

/home/ragarcia/anaconda3/envs/sfm/lib/python3.6/site-packages/numpy/core/_asarray.py:136: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray
  return array(a, dtype, copy=False, order=order, subok=True)


In [5]:
# Run this if you want to do reconstruction for the entire dataset
!opensfm create_tracks projects/mercedita

!opensfm reconstruct projects/mercedita

!opensfm mesh projects/mercedita

2020-09-02 13:36:58,448 INFO: reading features
2020-09-02 13:36:58,586 DEBUG: Merging features onto tracks
2020-09-02 13:36:58,794 DEBUG: Good tracks: 16095
2020-09-02 13:37:00,273 INFO: Starting incremental reconstruction
2020-09-02 13:37:00,605 INFO: Starting reconstruction with DSC_0025_75937cbe-1197-4b32-85c6-01888538517c.jpg and DSC_0026_a6c96465-3e9b-45a0-8157-bcc2e4bc8c97.jpg
2020-09-02 13:37:00,632 INFO: Two-view reconstruction inliers: 1999 / 1999
2020-09-02 13:37:00,724 INFO: Triangulated: 1999
2020-09-02 13:37:00,744 DEBUG: Ceres Solver Report: Iterations: 3, Initial cost: 1.820848e+03, Final cost: 1.815974e+03, Termination: CONVERGENCE
2020-09-02 13:37:00,870 DEBUG: Ceres Solver Report: Iterations: 3, Initial cost: 1.817139e+03, Final cost: 1.815770e+03, Termination: CONVERGENCE
2020-09-02 13:37:01,862 DEBUG: Ceres Solver Report: Iterations: 63, Initial cost: 4.545004e+01, Final cost: 2.688311e+01, Termination: CONVERGENCE
2020-09-02 13:37:01,867 INFO: Removed outliers: 0
2

2020-09-02 13:37:25,184 INFO: Re-triangulating
2020-09-02 13:37:26,548 DEBUG: Ceres Solver Report: Iterations: 25, Initial cost: 2.174498e+02, Final cost: 1.670040e+02, Termination: CONVERGENCE
2020-09-02 13:37:27,901 DEBUG: Ceres Solver Report: Iterations: 12, Initial cost: 1.803966e+02, Final cost: 1.774666e+02, Termination: CONVERGENCE
2020-09-02 13:37:27,918 INFO: Removed outliers: 2
2020-09-02 13:37:27,921 INFO: -------------------------------------------------------
2020-09-02 13:37:27,933 INFO: DSC_0012_da76f6eb-f7f8-4b55-ad99-038e95dbc4f7.jpg resection inliers: 839 / 857
2020-09-02 13:37:27,951 DEBUG: Ceres Solver Report: Iterations: 4, Initial cost: 9.785535e+01, Final cost: 8.364288e+01, Termination: CONVERGENCE
2020-09-02 13:37:27,951 INFO: Adding DSC_0012_da76f6eb-f7f8-4b55-ad99-038e95dbc4f7.jpg to the reconstruction
2020-09-02 13:37:28,010 DEBUG: Local bundle sets: interior 6  boundary 0  other 0
2020-09-02 13:37:28,450 DEBUG: Ceres Solver Report: Iterations: 11, Initial c

In [17]:
# Run this if you want to do reconstruction for a particular submodel
!opensfm create_tracks projects/mercedita/submodels/submodel_0000

!opensfm reconstruct projects/mercedita/submodels/submodel_0000

!opensfm mesh projects/mercedita/submodels/submodel_0000

2020-08-19 17:20:30,027 INFO: reading features
2020-08-19 17:20:30,086 DEBUG: Merging features onto tracks
2020-08-19 17:20:30,173 DEBUG: Good tracks: 6492
2020-08-19 17:20:31,282 INFO: Starting incremental reconstruction
2020-08-19 17:20:31,418 INFO: Starting reconstruction with DSC_0028_ecb7bab7-d7bd-4e03-b424-2aa624531942.jpg and DSC_0029_c0135ca8-d67d-49c6-8949-4bd1de71a10d.jpg
2020-08-19 17:20:31,443 INFO: Two-view reconstruction inliers: 1967 / 1967
2020-08-19 17:20:31,540 INFO: Triangulated: 1967
2020-08-19 17:20:31,560 DEBUG: Ceres Solver Report: Iterations: 3, Initial cost: 2.792961e+03, Final cost: 2.786900e+03, Termination: CONVERGENCE
2020-08-19 17:20:31,680 DEBUG: Ceres Solver Report: Iterations: 3, Initial cost: 2.788265e+03, Final cost: 2.786232e+03, Termination: CONVERGENCE
2020-08-19 17:20:32,932 DEBUG: Ceres Solver Report: Iterations: 101, Initial cost: 5.833665e+01, Final cost: 2.136713e+01, Termination: NO_CONVERGENCE
2020-08-19 17:20:32,939 INFO: Removed outliers: 

In order to visualize, run the following line:
`python -m http.server`

And go to http://localhost:8000/OpenSfM/viewer/reconstruction.html#file=/resil/CAP_SfM/projects/mercedita/reconstruction.json

Or, for submodels: http://localhost:8000/OpenSfM/viewer/reconstruction.html#file=/resil/CAP_SfM/projects/mercedita/submodels/submodel_0000/reconstruction.json

To reset reconstruction for a single model: `rm -rf camera_models.json exif/ matches/ reconstruction.json reference_lla.json tracks.csv features/ profile.log reconstruction.meshed.json reports/`

## Simple analysis

In [6]:
!opensfm export_ply --no-cameras --all projects/mercedita

In [3]:
def load_all_reconstructions(path):
    """
    Loads all reconstructions from an overall project path (not the path to the reconstruction files)
    """
    reconstructions_path = path + "reconstruction_files/"
    pcd = o3d.geometry.PointCloud()
    
    for file in os.listdir(reconstructions_path):
        pcd += o3d.io.read_point_cloud(reconstructions_path + file)
        
    return pcd

project_path = "projects/mercedita/"

# reconst_pcd = load_all_reconstructions(project_path)
reconst_pcd = o3d.geometry.PointCloud()
reconst_pcd += o3d.io.read_point_cloud(project_path + "reconstruction_files/reconstruction_0.ply")

pcd_coord = np.asarray(reconst_pcd.points)

## Refining the reconstruction using LIDAR point cloud
We can download a LIDAR point cloud from https://grid.nga.mil/grid. We can then refine our reconstruction with those external measurements.

In [4]:
from laspy.file import File
import numpy as np
import utm
import pymap3d as pm

# placeholder
filename = "projects/mercedita/lidar_las/FEMA/20180628_19Q682198/lidar_pointcloud/PC - Height Above Ground (HAG)/U_FEMA_PC_20200903.2157_1.las"

# Origin of our reconstruction, as given by the reference_lla.json (made from the reconstruction)
with open(project_path + "reference_lla.json", "r") as f:
    reference_lla = json.load(f)
    lat0=reference_lla["latitude"]
    lon0=reference_lla["longitude"]
    alt0=reference_lla["altitude"]
    
    
def pcd_from_las(filename, lat0, lon0, alt0):
    """
    Creates an Open3D point cloud object from a LAS file
    """
    inFile = File(filename, mode='r')
    
    # Scaling all dimensions
    x_dimension = inFile.X
    scale = inFile.header.scale[0]
    offset = inFile.header.offset[0]
    x_dimension = x_dimension*scale + offset

    y_dimension = inFile.Y
    scale = inFile.header.scale[1]
    offset = inFile.header.offset[1]
    y_dimension = y_dimension*scale + offset

    z_dimension = inFile.Z
    scale = inFile.header.scale[2]
    offset = inFile.header.offset[2]
    z_dimension = z_dimension*scale + offset
    
    # Converting to geodetic
    lat, lon = utm.to_latlon(x_dimension, y_dimension, 19, 'N')
    
    # Converting to ENU
    e, n, u = pm.geodetic2enu(lat, lon, z_dimension, lat0, lon0, alt0)
    
    # Converting to Open3D
    pcd_array = np.array([e, n, u]).T
    pcd = o3d.geometry.PointCloud()
    pcd.points = o3d.utility.Vector3dVector(pcd_array)
    
    return pcd
    

lidar_pcd = pcd_from_las(filename, lat0, lon0, alt0)
joint_pcd = reconst_pcd + lidar_pcd
o3d.visualization.draw_geometries([joint_pcd])

# print(point_records)
# for spec in inFile.point_format:
#     print(spec.name)
    
# z_coord = inFile.Z
# plt.hist(z_coord)
# plt.show()

# print(inFile.header.scale)

In [47]:
# pre-processing of the reconstructed point cloud 
reconst_pcd, idx = reconst_pcd.remove_statistical_outlier(nb_neighbors=20,
                                                    std_ratio=2.0)

In [50]:
import copy
# from http://www.open3d.org/docs/release/tutorial/Basic/icp_registration.html
def draw_registration_result(source, target, transformation):
    source_temp = copy.deepcopy(source)
    target_temp = copy.deepcopy(target)
    source_temp.paint_uniform_color([1, 0.706, 0])
    target_temp.paint_uniform_color([0, 0.651, 0.929])
    source_temp.transform(transformation)
    o3d.visualization.draw_geometries([source_temp, target_temp])

# implementing the ICP algorithm
trans_init = np.eye(4)
threshold = 50
draw_registration_result(reconst_pcd, lidar_pcd, trans_init)


print("Initial alignment")
evaluation = o3d.registration.evaluate_registration(reconst_pcd, lidar_pcd, threshold, trans_init)
print(evaluation)

Initial alignment
registration::RegistrationResult with fitness=9.451726e-01, inlier_rmse=1.967211e+01, and correspondence_set size of 8654
Access transformation to get result.


In [51]:
print("Apply point-to-point ICP")
reg_p2p = o3d.registration.registration_icp(
        reconst_pcd, lidar_pcd, threshold, trans_init,
        o3d.registration.TransformationEstimationPointToPoint())
print(reg_p2p)
print("Transformation is:")
print(reg_p2p.transformation)
draw_registration_result(reconst_pcd, lidar_pcd, reg_p2p.transformation)

Apply point-to-point ICP
registration::RegistrationResult with fitness=9.525994e-01, inlier_rmse=7.688785e+00, and correspondence_set size of 8722
Access transformation to get result.
Transformation is:
[[ 9.99130527e-01  6.29380319e-03 -4.12137974e-02 -5.22765919e+00]
 [-6.88801488e-03  9.99874143e-01 -1.42916912e-02 -9.07581702e+00]
 [ 4.11186612e-02  1.45631462e-02  9.99048132e-01 -2.02679649e+01]
 [ 0.00000000e+00  0.00000000e+00  0.00000000e+00  1.00000000e+00]]


## Georeferencing of the image
Steps:
- Export the reconstruction as ply
- Examine the bounds of the reconstructed points
- Perform the transformation

**Conclusion:** the distribution of point coordinates is much more reasonable! Might be a good candidate for georeferencing.

Now we will add the GCPs as done in Geoprocessing with Python.

In [5]:
# Origin of our reconstruction, as given by the reference_lla.json (made from the reconstruction)
with open(project_path + "reference_lla.json", "r") as f:
    reference_lla = json.load(f)
    latitude=reference_lla["latitude"]
    longitude=reference_lla["longitude"]
    altitude=reference_lla["altitude"]

# This is the json file that contains the reconstructed feature points
with open(project_path + "reconstruction.json", "r") as f:
    reconstructions = json.load(f)
    
# reading the tracks csv
tracks = pd.read_csv(project_path + "tracks.csv", sep="\t", skiprows=1, names=["image_name", "track_id", "feature_id", "x", "y", "s", "R", "G", "B"])

In [10]:
if not os.path.isdir(project_path + "geotiff/"):
    os.mkdir(project_path + "geotiff/")

    
lidar_pcd = pcd_from_las(filename, lat0, lon0, alt0)   
# here we put in the actual GCPs
for reconst in reconstructions:
    for shot in reconst["shots"]:
        # some housekeeping
        shot_name = shot.split(".")[0]
        img = cv2.imread(project_path + "images/"+shot)
        shape = img.shape
        
        # get the features from the image and match them with their corresponding reconstructed features
        reconst_ids = list(map(int, reconst["points"].keys()))
        tracks_shot = tracks[(tracks["image_name"] == shot) & (tracks["track_id"].isin(reconst_ids))]
        norm_shot = tracks_shot[["x", "y"]].values
        denorm_shot = denormalized_image_coordinates(norm_shot, shape[1], shape[0])
        reconst_shot = np.array([reconst["points"][str(point)]["coordinates"] for point in tracks_shot["track_id"]])
        
        # filtering out all outliers
        pcd = o3d.geometry.PointCloud()
        pcd.points = o3d.utility.Vector3dVector(reconst_shot)
        _, inliers = pcd.segment_plane(distance_threshold=100,
                                         ransac_n=3,
                                         num_iterations=1000)
        denorm_shot = denorm_shot[inliers, :]
        reconst_shot = reconst_shot[inliers, :]
        
        idx = random.sample(range(len(inliers)), 20)
        denorm_shot = denorm_shot[idx, :]
        reconst_shot = reconst_shot[idx, :]
        
        # converting all ENU coordinates to lat/lon coordinates
        geo_shot = np.array([enu2geodetic(reconst_shot[i, 0],reconst_shot[i, 1],reconst_shot[i, 2],latitude,longitude,altitude) for i in range(reconst_shot.shape[0])])  
        
        # creating a copy of the jpg as a tif
        orig_fn = project_path + "images/" + shot
        fn = project_path + "geotiff/" + shot_name + ".tif"
        orig_ds = gdal.Open(orig_fn)
        gdal.GetDriverByName('GTiff').CreateCopy(fn, orig_ds)
        
        # updating the geotiff to include the gcps
        ds = gdal.Open(fn, gdal.GA_Update)
        sr = osr.SpatialReference()
        sr.SetWellKnownGeogCS('WGS84')
        
        gcps = [gdal.GCP(geo_shot[i, 1], geo_shot[i, 0], 0, int(denorm_shot[i, 0]), int(denorm_shot[i, 1])) for i in range(geo_shot.shape[0])]
        
        ds.SetGCPs(gcps, sr.ExportToWkt())
        
        ds = None

In [7]:
# TODO: do the same but for all of the translated points
# defining variables
trans_init = np.eye(4)
threshold = 10

# loading the lidar point cloud
lidar_pcd = pcd_from_las(filename, lat0, lon0, alt0)   


# creating a folder for the geotiff files if they do not exist
if not os.path.isdir(project_path + "geotiff/"):
    os.mkdir(project_path + "geotiff/")

for reconst_idx, reconst in enumerate(reconstructions):
    # ids of the points in the reconstruction
#     reconst_ids = list(map(int, reconst["points"].keys()))
    reconst_ids = list(reconst["points"].keys())
    
    # turning the json reconstruction into a point cloud
    reconst_array = np.array([reconst["points"][point_id]["coordinates"] for point_id in reconst_ids])
    reconst_pcd = o3d.geometry.PointCloud()
    reconst_pcd.points = o3d.utility.Vector3dVector(reconst_array)

    # downsampling statistical outliers
    reconst_pcd, idx_inliers = reconst_pcd.remove_statistical_outlier(nb_neighbors=20,
                                                    std_ratio=1.0)
    
    # create dictionary that relates reconstruction ids to ply indeces
    reconst_ids = [reconst_ids[i] for i in idx_inliers]
    reconst_dict = {int(reconst_id):idx for idx, reconst_id in enumerate(reconst_ids)}
    
    # realigning using ICP
    reg_p2p = o3d.registration.registration_icp(
        reconst_pcd, lidar_pcd, threshold, trans_init,
        o3d.registration.TransformationEstimationPointToPoint(),
        o3d.registration.ICPConvergenceCriteria(max_iteration = 2000))
    
    reconst_pcd = reconst_pcd.transform(reg_p2p.transformation)
    
    
    for shot in reconst["shots"]:
        # some housekeeping
        shot_name = shot.split(".")[0]
        img = cv2.imread(project_path + "images/"+shot)
        shape = img.shape
        
        # get the features from the image and match them with their corresponding reconstructed features
        tracks_shot = tracks[(tracks["image_name"] == shot) & (tracks["track_id"].isin(reconst_dict.keys()))]
        norm_shot = tracks_shot[["x", "y"]].values
        denorm_shot = denormalized_image_coordinates(norm_shot, shape[1], shape[0])
        
        for track_id in tracks_shot["track_id"]:
            try:
                reconst_dict[track_id]
                reconst_pcd.points[reconst_dict[track_id]]
            except:
                print(reconst_dict[track_id])
            
        reconst_shot = np.array([reconst_pcd.points[reconst_dict[track_id]] for track_id in tracks_shot["track_id"]])
        
#         pcd = o3d.geometry.PointCloud()
#         pcd.points = o3d.utility.Vector3dVector(reconst_shot)
#         pcd = pcd.voxel_down_sample(voxel_size=0.10)
        
        idx = np.random.randint(len(tracks_shot), size=int(len(tracks_shot)*.10))
        reconst_shot = reconst_shot[idx, :]
        
        # converting all ENU coordinates to lat/lon coordinates
        geo_shot = np.array([enu2geodetic(reconst_shot[i, 0],reconst_shot[i, 1],reconst_shot[i, 2],latitude,longitude,altitude) for i in range(reconst_shot.shape[0])])  
        
        # creating a copy of the jpg as a tif
        orig_fn = project_path + "images/" + shot
        fn = project_path + "geotiff/" + shot_name + ".tif"
        orig_ds = gdal.Open(orig_fn)
        gdal.GetDriverByName('GTiff').CreateCopy(fn, orig_ds)
        
        # updating the geotiff to include the gcps
        ds = gdal.Open(fn, gdal.GA_Update)
        sr = osr.SpatialReference()
        sr.SetWellKnownGeogCS('WGS84')
        
        gcps = [gdal.GCP(geo_shot[i, 1], geo_shot[i, 0], 0, int(denorm_shot[i, 0]), int(denorm_shot[i, 1])) for i in range(geo_shot.shape[0])]
        
        ds.SetGCPs(gcps, sr.ExportToWkt())
        
        ds = None

In [36]:
len(tracks_shot)

1944